# Modelling and Deployment using MLOps 

Now that we have audio input data & corresponding labels in an array format, it is easier to consume and apply Natural language processing techniques. We can convert audio files labels into integers using label Encoding or One Hot Vector Encoding for machines to learn. The labeled dataset will help us in the neural network model output layer for predicting results. These help in training & validation datasets into nD array.
At this stage, we apply other pre-processing techniques like dropping columns, normalization, etc. to conclude our final training data for building models. Moving to the next stage of splitting the dataset into train, test, and validation is what we have been doing for other models. The below diagram is a generic representation of the Convolution Neural Network.
We can leverage CNN, RNN, LSTM, etc. deep neural algorithms to build and train the models for speech applications like speech recognition, emotion recognition, music genre classification, voice biometric, and many more. The model trained with the standard size few seconds audio chunk transformed into an array of n dimensions with the respective labels will result in predicting output labels for test audio input. As output labels will vary beyond binary, we are talking about building a multi-class label classification method.


In [4]:
import pandas as pd
import os,sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from deep_learner import DeepLearn
from modeling import Modeler

# Deep Learning Model

**objective**: Build a Deep learning model that converts speech to text.

In [2]:
swahili_df = pd.read_csv("../data/swahili.csv")
amharic_df = pd.read_csv("../data/amharic.csv")

In [3]:
swahili_df

,key,text,duration,rate,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete,3.9,44100,0.140494,0.315876,1866.409879,1589.100426,3359.724832,0.054397,-8.142890
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...,3.9,44100,0.162743,0.321692,1900.991630,1655.337451,3481.883645,0.049550,-6.665521
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...,3.9,44100,0.164615,0.355232,2063.848819,1775.474533,3859.115658,0.049307,-7.063959
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...,3.9,44100,0.168792,0.451515,1852.997999,1686.271162,3497.119321,0.048648,-1.036142
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya,3.9,44100,0.154860,0.319131,2039.119501,1727.509904,3764.721680,0.053856,-7.143945
5,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,kule abidjan raia wa jiji hilo,3.9,44100,0.135778,0.319711,1894.964967,1746.052445,3657.568359,0.044638,-6.545257
6,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,walipata fursa ya kutumia haki yao ya msingi,3.9,44100,0.130460,0.343494,2256.247714,1754.663927,4012.988713,0.076340,-7.412744
7,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,waziri mkuu wa zamani alasane watara,3.9,44100,0.162844,0.321559,1967.845186,1731.059926,3754.424650,0.054829,-6.871382
8,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,na rais aliyetangulia henry konan berdi,3.9,44100,0.143617,0.320329,1670.689876,1695.802935,3383.425735,0.038920,-7.546750
9,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,walichuana vikali na rais lauren bagbo,3.9,44100,0.152354,0.325879,1739.247669,1614.625697,3268.708095,0.042071,-7.587160
